In [26]:
#Where we Import 
from pyspark.sql.functions import col, lit, expr, when, to_timestamp
from pyspark.sql.types import *
from datetime import datetime
import time
import pyspark
from pyspark.sql import SparkSession
import random
import string
import pandas as pd
from pyspark.sql.functions import avg

In [2]:
##+===========================================================SET DATASETLENGTH HERE!!!
DataLength=1000000

In [3]:
#ORIGINAL DATASETS
#German Cities Data
german_cities_data = {
    "CityID": [1, 2, 3, 4, 5],
    "CityName": ["Berlin", "Hamburg", "Munich", "Cologne", "Frankfurt"],
    "Population": [3520000, 1790000, 1450000, 1060000, 733000],
    "Area": [891.3, 755.2, 310.7, 405.15, 500],
    "CulturalSites": [7, 6, 3, 5, 3]  
}

# French Cities Data
french_cities_data = {
    "CityID": [1, 2, 3, 4, 5],
    "CityName": ["Paris", "Marseille", "Lyon", "Toulouse", "Nice"],
    "Population": [2250000, 851000, 491000, 447000, 344000],
    "Area": [250, 240.6, 47.87, 118.3, 71.9],      
    "CulturalSites": [7, 7, 1, 4, 3]
}

In [5]:

# Function to generate random city name
def generate_city_name(length):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))

# Function to generate random data for cities
def generate_city_data(num_cities):
    cities_data = []
    for i in range(num_cities):
        city_name = generate_city_name(random.randint(4, 6))
        population = random.randint(350, 5000000)
        area = round(random.uniform(170, 5000), 2)
        tourist_visits = random.randint(3000000, 6000000)
        cultural_sites = random.randint(2, 7)
        cities_data.append({
            "CityID": i + 1,
            "CityName": city_name,
            "Population": population,
            "Area": area,
            "TouristVisits": tourist_visits,
            "CulturalSites": cultural_sites
        })
    return cities_data



In [6]:
# # Augment premade German cities data
#This all just adds the random data to the premade dataframes
# german_cities_augmented = german_cities_data.copy()
# german_cities_augmented["CityID"].extend(range(6, DataLength+6))
# german_cities_augmented["CityName"].extend([city_data["CityName"] for city_data in generate_city_data(DataLength)])
# german_cities_augmented["Population"].extend([city_data["Population"] for city_data in generate_city_data(DataLength)])
# german_cities_augmented["Area"].extend([city_data["Area"] for city_data in generate_city_data(DataLength)])
# german_cities_augmented["CulturalSites"].extend([city_data["CulturalSites"] for city_data in generate_city_data(DataLength)])

# # Augment premade French cities data
# french_cities_augmented = french_cities_data.copy()
# french_cities_augmented["CityID"].extend(range(6, DataLength+6))
# french_cities_augmented["CityName"].extend([city_data["CityName"] for city_data in generate_city_data(DataLength)])
# french_cities_augmented["Population"].extend([city_data["Population"] for city_data in generate_city_data(DataLength)])
# french_cities_augmented["Area"].extend([city_data["Area"] for city_data in generate_city_data(DataLength)])
# french_cities_augmented["CulturalSites"].extend([city_data["CulturalSites"] for city_data in generate_city_data(DataLength)])


In [ ]:
# #MAKE DATAFRAMES using pandas 
# df_german_cities = pd.DataFrame(german_cities_data)
# df_french_cities = pd.DataFrame(french_cities_data)

In [7]:
# # Save to CSV files: ===================== !GRADER YOU WILL NEED TO UNCOMMENT THIS TO RUN CODE ON YOUR MACHINE!

# df_german_cities.to_csv('german_cities.csv', index=False)
# df_french_cities.to_csv('french_cities.csv', index=False) 

In [8]:
#Calling the paths to the saved Dataframes
frenchpath=r"C:\Users\alexw\Desktop\BDG3_FTP\Group_Assignment\french_cities.csv"
Germpath=r"C:\Users\alexw\Desktop\BDG3_FTP\Group_Assignment\german_cities.csv"

In [9]:
#Establishing A spark Session
spark = SparkSession.builder \
    .appName("Assignment 1") \
    .getOrCreate()

In [10]:
#This will set and apply a schema for the French Cities dataset
F_schema = StructType([
    StructField("CityID", IntegerType(), nullable=False),
    StructField("CityName", StringType(), nullable=False),
    StructField("Population", IntegerType(), nullable=False),
    StructField("Area", DoubleType(), nullable=False),
    StructField("CulturalSites", IntegerType(), nullable=False)
])

french_cities = spark.read.format("csv").load(frenchpath, schema=F_schema)

In [11]:
#This will set a schema for the German Cities Dataset
G_schema = StructType([
    StructField("CityID", IntegerType(), nullable=False),
    StructField("CityName", StringType(), nullable=False),
    StructField("Population", IntegerType(), nullable=False),
    StructField("Area", DoubleType(), nullable=False),
    StructField("CulturalSites", IntegerType(), nullable=False)
])

german_cities = spark.read.format("csv").load(Germpath, schema=G_schema)

In [19]:
#This is just to drop the first row of each dataframe which for some reason is just the column names and null values
german_cities=german_cities.dropna()
french_cities=french_cities.dropna()

In [22]:
#ALEX Q2
#Pyspark Dataframe Operation
#Here we are selecting both data frames and ordering them by area. 
#We can see French cities are a bit larger by area
germ_city_sizes = german_cities.select("CityName", "Area", "Population").orderBy(col("Area").desc()).show()
french_city_sizes = french_cities.select("CityName", "Area", "Population").orderBy(col("Area").desc()).show()
#============================================

+--------+-------+----------+
|CityName|   Area|Population|
+--------+-------+----------+
|    Brpk| 5000.0|    191432|
|    ZimV|4999.98|   3390964|
|  AteeRA|4999.98|   1671934|
|  cBgzzc|4999.98|   1289866|
|    MAqq|4999.98|   3186545|
|  fKfxif|4999.96|   1210617|
|    Iuww|4999.95|   2500533|
|   HpzNW|4999.95|   3627489|
|  HYUuaO|4999.94|   2027121|
|    drFn|4999.94|   4974197|
|   foZww|4999.94|   4023043|
|   BCEeu|4999.94|    939649|
|    Mycw|4999.93|   3362677|
|   tnOVK|4999.92|   1104528|
|    ENaA|4999.92|   2308746|
|  sVBbsm|4999.91|   4239354|
|   CJowj|4999.91|   3651452|
|  qIObjO| 4999.9|   2416419|
|   eUZMg|4999.89|   2817431|
|   cwoBa|4999.89|   3540298|
+--------+-------+----------+
only showing top 20 rows

+--------+-------+----------+
|CityName|   Area|Population|
+--------+-------+----------+
|  GbmkVy| 5000.0|    185132|
|   KThrN| 5000.0|   4646191|
|    NNzT| 5000.0|   3565660|
|  awYDYU|4999.98|    580237|
|    DkIJ|4999.98|   4201978|
|  duXZXo|4999

In [23]:
#Alex Q2
#SQL Version
#Making temp views so I can use SQL
german_cities.createOrReplaceTempView("german_cities")
french_cities.createOrReplaceTempView("french_cities")

# German City Call
germ_city_sizes = spark.sql("""
    
    SELECT CityName, Area, Population 
    FROM german_cities 
    ORDER BY Area DESC

""")
germ_city_sizes.show()

#French City Call
french_city_sizes = spark.sql("""
    
    SELECT CityName, Area, Population 
    FROM french_cities 
    ORDER BY Area DESC

""")

# Show the result for French cities
french_city_sizes.show()

+--------+-------+----------+
|CityName|   Area|Population|
+--------+-------+----------+
|    Brpk| 5000.0|    191432|
|    ZimV|4999.98|   3390964|
|  AteeRA|4999.98|   1671934|
|  cBgzzc|4999.98|   1289866|
|    MAqq|4999.98|   3186545|
|  fKfxif|4999.96|   1210617|
|   HpzNW|4999.95|   3627489|
|    Iuww|4999.95|   2500533|
|  HYUuaO|4999.94|   2027121|
|    drFn|4999.94|   4974197|
|   foZww|4999.94|   4023043|
|   BCEeu|4999.94|    939649|
|    Mycw|4999.93|   3362677|
|    ENaA|4999.92|   2308746|
|   tnOVK|4999.92|   1104528|
|   CJowj|4999.91|   3651452|
|  sVBbsm|4999.91|   4239354|
|  qIObjO| 4999.9|   2416419|
|   cwoBa|4999.89|   3540298|
|   sFqMf|4999.89|   1819157|
+--------+-------+----------+
only showing top 20 rows

+--------+-------+----------+
|CityName|   Area|Population|
+--------+-------+----------+
|   KThrN| 5000.0|   4646191|
|  GbmkVy| 5000.0|    185132|
|    NNzT| 5000.0|   3565660|
|  duXZXo|4999.98|   1342306|
|  awYDYU|4999.98|    580237|
|    DkIJ|4999

In [25]:
#ALEX Q6
#Pyspark Dataframe operation
#For this question I am using
print("Germany")
german_cities.select(
    avg("CityID").alias("Avg_CityID"),
    avg("Population").alias("Avg_Population"),
    avg("Area").alias("Avg_Area"),
    avg("CulturalSites").alias("Avg_CulturalSites")
).show()

print("France")
french_cities.select(
    avg("CityID").alias("Avg_CityID"),
    avg("Population").alias("Avg_Population"),
    avg("Area").alias("Avg_Area"),
    avg("CulturalSites").alias("Avg_CulturalSites")
).show()


#As we can see they are similar sizes, with Germany having slightly more cultural sites and population
#This makes sense that both would be similar as they are randomly generated via the same parameters


Germany
+----------+------------------+------------------+-----------------+
|Avg_CityID|    Avg_Population|          Avg_Area|Avg_CulturalSites|
+----------+------------------+------------------+-----------------+
|  500003.0|2499948.9589232053|2585.7041581891945|4.499701501492493|
+----------+------------------+------------------+-----------------+

France
+----------+------------------+------------------+-----------------+
|Avg_CityID|    Avg_Population|          Avg_Area|Avg_CulturalSites|
+----------+------------------+------------------+-----------------+
|  500003.0|2499667.3588442057|2584.0123058284817| 4.50078249608752|
+----------+------------------+------------------+-----------------+



In [29]:
#Alex Q6
#SQL Version

#Using the previously created views in Q2
#German query
avg_german_stats = spark.sql("""
    
    SELECT AVG(CityID) AS Avg_CityID,
           AVG(Population) AS Avg_Population,
           AVG(Area) AS Avg_Area,
           AVG(CulturalSites) AS Avg_CulturalSites
    FROM german_cities
    
""")

avg_german_stats.show()

# French Query
avg_french_stats = spark.sql("""

    SELECT AVG(CityID) AS Avg_CityID,
           AVG(Population) AS Avg_Population,
           AVG(Area) AS Avg_Area,
           AVG(CulturalSites) AS Avg_CulturalSites
    FROM french_cities
    
""")
avg_french_stats.show()


+----------+------------------+------------------+-----------------+
|Avg_CityID|    Avg_Population|          Avg_Area|Avg_CulturalSites|
+----------+------------------+------------------+-----------------+
|  500003.0|2499948.9589232053|2585.7041581891945|4.499701501492493|
+----------+------------------+------------------+-----------------+

+----------+------------------+------------------+-----------------+
|Avg_CityID|    Avg_Population|          Avg_Area|Avg_CulturalSites|
+----------+------------------+------------------+-----------------+
|  500003.0|2499667.3588442057|2584.0123058284817| 4.50078249608752|
+----------+------------------+------------------+-----------------+

